In [ ]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import box

# Load the Norway shapefile
norway = gpd.read_file("../data/gadm41_NOR_shp/gadm41_NOR_0.shp")

# Reproject to UTM zone 33N (EPSG:32633) for 1x1 km grid creation
norway = norway.to_crs(epsg=32633)

# Function to create a 1km x 1km grid
def create_grid(bounds, cell_size=1000):
    xmin, ymin, xmax, ymax = bounds
    grid_cells = []
    
    for x in range(int(xmin), int(xmax), cell_size):
        for y in range(int(ymin), int(ymax), cell_size):
            grid_cells.append(box(x, y, x + cell_size, y + cell_size))
    
    return gpd.GeoDataFrame(geometry=grid_cells, crs="EPSG:32633")

# Create grid over the bounding box of Norway
grid = create_grid(norway.total_bounds)

# Clip grid to Norway borders (optional but cleaner)
grid = gpd.overlay(grid, norway, how='intersection')

# Compute centroids in UTM (projected) coordinates
grid["centroid"] = grid.geometry.centroid

# Create a new GeoDataFrame with centroids, and convert to WGS84 for lat/lon
centroids = grid.copy()
centroids = centroids.set_geometry("centroid").to_crs(epsg=4326)

# Extract latitude and longitude
centroids["latitude"] = centroids.geometry.y
centroids["longitude"] = centroids.geometry.x

# Final DataFrame with centroid lat/lon and polygon geometry
grid["latitude"] = centroids["latitude"]
grid["longitude"] = centroids["longitude"]

# Keep only needed columns
grid_df = grid[["geometry", "latitude", "longitude"]]

# Done: This is your 1km grid with centroids
grid_df.head()


In [ ]:
# Definitive function from Sigma2

import geopandas as gdp
import apndas as pd
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import box

norway = gpd.read_file("../data/gadm41_NOR_shp/gadm41_NOR_0.shp")

# Reproject to UTM zone 33N (EPSG:32633) for 1x1 km grid creation
norway = norway.to_crs(epsg=32633)

# simplify geometry (tolerance in meters)
norway_simple = norway.copy()
norway_simple["geometry"] = norway.simplify(tolerance=500, preserve_topology = True)

# 4. Compare original vs simplified
fig, ax = plt.subplots(1, 2, figsize=(14, 6))
norway.plot(ax=ax[0], edgecolor='black')
ax[0].set_title("Original Norway Geometry")
norway_simple.plot(ax=ax[1], edgecolor='red')
ax[1].set_title("Simplified Norway Geometry")
plt.tight_layout()
plt.show()

# 5. Function to create a 1x1 km grid
def create_grid(bounds, cell_size=1000):
    xmin, ymin, xmax, ymax = bounds
    x_coords = np.arange(xmin, xmax, cell_size)
    y_coords = np.arange(ymin, ymax, cell_size)

    polygons = [box(x, y, x + cell_size, y + cell_size) for x in x_coords for y in y_coords]
    return gdp.GeoDataFrame(geometry=polygons, crs="EPSG:32633")

# 6. Create a grid and clip it using simplified borders
grid = create_grid(norway.total_bounds)
print("grid created")
grid = gpd.overlay(grid, norway_simple, how = 'intersection') # much faster

# 7. Calculate centroids and convert to lat/lon (WGS84)
grid["centroid"] = grid.geometry.centroid
centroids = grid.set_geometry("centroid").to_crs(epsg = 4326)
grid["latitude"] = centroids.geometry.y
grid["longitude"] = centroids.geometry.x

# 8 keep only necessary info
grid_df = grid[["geometry","latitude","longitude"]]

grid_df

In [ ]:
import xarray as xr
import numpy as np
from sklearn.preprocessing import MinMaxScaler

def read_netcdf_stack(file_paths, latitude, longitude, grid):
    patches = []

    for file_path in file_paths:
        data = xr.open_dataset(file_path)
        variable = list(data.data_vars.keys())[-1]

        lat_idx = np.argmin(np.abs(data['lat'].values - latitude))
        lon_idx = np.argmin(np.abs(data['lon'].values - longitude))

        patch = data[variable].isel(
            lat=slice(lat_idx - grid, lat_idx + grid),
            lon=slice(lon_idx - grid, lon_idx + grid)
        ).fillna(0).values  # Convert to NumPy

        print(f"Shape of patch for {file_path} {variable}: {patch.shape}")

        # Remove extra dimension if necessary
        if patch.shape[0] == 1:
            patch = patch[0]

        patches.append(patch)  # Add raw patch (not scaled)

    # Stack along last axis (channels dimension)
    stacked_patches = np.stack(patches, axis=-1)

    return stacked_patches



In [ ]:
path_altitude = r"C:\Users\Raul\Desktop\TFM\CHELSA\DEM\dem_latlong.nc"

variables = ['gdd5', 'prsd', 'bio12d', 'swe', 'bio01d', 'bio04d', 'cdd', 'fd', 'bio15d', 'scd']
file_paths = [f"CHELSA/1991-2020/chelsav2/EUR11/obs/annual/V2.1/{var}/CHELSA_EUR11_obs_{var}_2000_V.2.1.nc" for var in variables]
file_paths.append(path_altitude)

In [ ]:
resolution = 32 # resolutions = [8,16,32]
stacked_patches_list = []
#x = 100
for i in range(gbif_assemblages.shape[0]): # x
    sample =  gbif_assemblages.iloc[i]
    file_paths = [f"CHELSA\\1991-2020\\chelsav2\\EUR11\\obs\\annual\\V2.1\\{var}\\CHELSA_EUR11_obs_{var}_{sample.year}_V.2.1.nc" for var in variables]
    file_paths.append(path_altitude)
        
    # Call the function to get the stacked patches
    stacked_patches = read_netcdf_stack(file_paths = file_paths, 
                                            latitude = sample.latitude, 
                                            longitude = sample.longitude, 
                                            grid = resolution) # better to use multiples of 64
        
    stacked_patches_list.append(stacked_patches)
    
    print(f'Sample {i} correctly stacked and added to the list')
    print('\n\n')

In [ ]:
# Save DataFrame as a Pickle file
with open(f'Data\\Full_Scale\\vectorized__climatic_maps_ALL-NORWAY_2018_{resolution*2}.pkl', "wb") as file_2:
    pickle.dump(gbif_assemblages_climatic, file_2)